In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import numpy
import xgboost as xgb
import statistics as st
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import random

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_df = pd.read_csv("train.csv")

In [3]:
valid_df = pd.read_csv("validation.csv")

In [ ]:
test_df = pd.read_csv("test.csv")

In [4]:
def test_results(threshold, y_p, real_clicks, payprice):
    
    pred_clicks = y_p > threshold # predicted clicks with threshold
    impressions = sum(pred_clicks)
    clicked = sum((pred_clicks + real_clicks) == 2)
    total_cost = sum(valid_df.payprice[clicks_0 == 1])
    
    ctr = clicked/impressions
    cpc = total_cost/clicked
    
    
    print("CTR is ", ctr, " impressions ", impressions, " clicks ", clicked, " total cost ", total_cost, " CPC ", cpc)
    
    return pred_clicks

In [5]:
def test_results_budget(clicks, payprice, real_clicks):
    
    sum_ones = 0
    spent = 0
    click_total = 0

    for i in range(0,len(clicks),3):
        if (clicks[i] == 1) & (spent < 6250):
            click_total = click_total + 1
            spent = spent + payprice[i]
            if real_clicks[i] == 1:
                sum_ones = sum_ones + 1

        if spent > 6250:
            break

    cpc = spent/sum_ones
    ctr_1 = sum_ones/click_total
    print("Sum payed for the clicked ones ", spent, " successful clicks ", sum_ones, " out of ", click_total)
    print("Cost per Click ", cpc)
    print("CTR ", ctr_1)

## clean data

In slotformat map Na to 10 and in adexchange map null tu 0

In [6]:
for index, c in train_df.iterrows():  
    if c.slotformat in "Na":
        train_df.set_value(index, 'slotformat', 10)
        
d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
train_df['adexchange']=train_df['adexchange'].map(d)

In [7]:
for index, c in valid_df.iterrows():  
    if c.slotformat in "Na":
        valid_df.set_value(index, 'slotformat', 10)

d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
valid_df['adexchange']=valid_df['adexchange'].map(d)

In [ ]:
for index, c in test_df.iterrows():  
    if c.slotformat in "Na":
        test_df.set_value(index, 'slotformat', 10)

d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
test_df['adexchange']=test_df['adexchange'].map(d)

## Constant bidding

In [8]:
const = train_df.bidprice.get_values() #[300, 294, 238, 227, 241, 277, 254, 249]
const = np.unique(const)
const = [c+1 for c in const]
const = np.sort(const)
print(const)

[228 239 242 250 255 278 295 301]


doing for training set (ctr) and validation set (ctr_v)

In [9]:
ctr = np.arange(0, len(const), dtype=np.float)
cpc = np.arange(0, len(const), dtype=np.float)
ctr_v = np.arange(0, len(const), dtype=np.float)
cpc_v = np.arange(0, len(const), dtype=np.float)
total = np.arange(0, len(const), dtype=np.float)
total_v = np.arange(0, len(const), dtype=np.float)
clicks_c = np.arange(0, len(const), dtype=np.float)
clicks_c_v = np.arange(0, len(const), dtype=np.float)
i = 0

for c in const:
    #filt is clicks/non-clicks for impressions
    filt = train_df.click[train_df.bidprice < c]
    filt_v = valid_df.click[valid_df.bidprice < c]
    
    # clickes imp / total imp
    ctr[i] = sum(filt)*100/len(filt)
    ctr_v[i] = sum(filt_v)*100/len(filt_v)
    
    # clicked imp
    clicks_c[i] = sum(filt)
    clicks_c_v[i] = sum(filt_v)  
    
    # impressions
    total[i] = len(filt)
    total_v[i] = len(filt_v)
    
    # sum payprice of imp / num of clicks
    cpc[i] = sum(train_df.payprice[train_df.bidprice < c])/clicks_c[i]
    cpc_v[i] = sum(valid_df.payprice[valid_df.bidprice < c])/clicks_c_v[i]

    i = i + 1
    
max_index = np.argmax(ctr)
max_index_v = np.argmax(ctr_v)
print("The highest CTR for constant bidding is for ", const[max_index], " with CTR = ", ctr[max_index], \
      "% and CPC = ", cpc[max_index])

print("Numer of impressions for training set", total[max_index], " with ", clicks_c[max_index] ,\
      " clicks and total cost", (cpc[max_index])*clicks_c[max_index],"\n")

print("The highest CTR for constant bidding  in valid. set is for ", const[max_index_v], " with CTR = ", \
      ctr_v[max_index_v], "% and CPC = ", cpc_v[max_index_v])

print("Numer of impressions for validation set", total_v[max_index_v], " with ", clicks_c_v[max_index_v] ,\
      " clicks and total cost", (cpc_v[max_index_v])*clicks_c_v[max_index_v])

The highest CTR for constant bidding is for  228  with CTR =  0.0981850004625 % and CPC =  108275.126812
Numer of impressions for training set 281102.0  with  276.0  clicks and total cost 29883935.0 

The highest CTR for constant bidding  in valid. set is for  228  with CTR =  0.1112276353 % and CPC =  95139.7714286
Numer of impressions for validation set 31467.0  with  35.0  clicks and total cost 3329892.0


In [10]:
clicks_const = valid_df.bidprice < const[max_index_v]
test_results_budget(clicks_const, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6320  successful clicks  0  out of  56
Cost per Click  inf
CTR  0.0


## Random bidding

In [11]:
# 74 is the difference
train_rb = train_df
valid_rb = valid_df
upper = range(10, 80, 10) # rounded up range obtainded from max_bidprice - min_bidprice

In [12]:
random_bid_basics = np.random.rand(len(train_rb), len(upper))
name = ["bid1", "bid2", "bid3", "bid4", "bid5", "bid6", "bid7"]
ctr_r = np.arange(0, len(upper), dtype=np.float)
cpc_r = np.arange(0, len(upper), dtype=np.float)
bottom = 227
i = 0

for u in upper:
    train_rb[name[i]] = random_bid_basics[:,i]*u + bottom
   
    
    filt = train_rb.click[train_rb.bidprice < train_rb[name[i]]]
    ctr_r[i] = sum(filt)*100/len(filt)
    cpc_r[i] = sum(train_rb.payprice[train_rb.bidprice < train_rb[name[i]]])/sum(filt)
    print("num of clicks: ", sum(filt), "total: ", len(filt), " cpc ", cpc_r[i], " ctr ", ctr_r[i])
    
    i = i+1

num of clicks:  276 total:  281102  cpc  108275.126812  ctr  0.0981850004625
num of clicks:  408 total:  506732  cpc  116741.938725  ctr  0.0805159334717
num of clicks:  487 total:  668154  cpc  122761.683778  ctr  0.0728873882368
num of clicks:  565 total:  775853  cpc  120083.244248  ctr  0.072823073443
num of clicks:  611 total:  842939  cpc  119186.409165  ctr  0.0724844858287
num of clicks:  684 total:  919735  cpc  115207.710526  ctr  0.0743692476637
num of clicks:  739 total:  990367  cpc  114415.313938  ctr  0.0746188029286


In [13]:
max_index_r = np.argmax(ctr_r)
print("The highest CTR for random bidding is for upper bound at ", upper[max_index_r], \
      " with CTR = ", ctr_r[max_index_r], "% with CPC = ", cpc_r[max_index_r])
max_index_cpc_r = np.argmax(cpc_r)
print("The highest CPC for random bidding is for upper bound at ", upper[max_index_cpc_r], \
      " with CPC = ", cpc_r[max_index_cpc_r]," and CTR = ", ctr_r[max_index_cpc_r], "%")


The highest CTR for random bidding is for upper bound at  10  with CTR =  0.0981850004625 % with CPC =  108275.126812
The highest CPC for random bidding is for upper bound at  30  with CPC =  122761.683778  and CTR =  0.0728873882368 %


In [14]:
random_bid_basics_v = np.random.rand(len(valid_rb), 1)
valid_rb["rnd_bid"] = random_bid_basics_v * upper[max_index] + bottom

auction = valid_rb.bidprice < valid_rb["rnd_bid"]
clicks_rand = valid_rb.click[valid_rb.bidprice < valid_rb["rnd_bid"]]

ctr_r_v = sum(clicks_rand)*100/len(clicks_rand)
cpc_r_v = sum(valid_rb.payprice[valid_rb.bidprice < valid_rb["rnd_bid"]])/sum(clicks_rand)

print("The CTR for random bidding on validation set is for upper bound at ", upper[max_index], \
      " is ", ctr_r_v ,"%")

print("The CPC for random bidding on validation set is for upper bound at ", upper[max_index_r], \
      " is ", cpc_r_v)

print("Total cost for valid set is ", sum(valid_rb.payprice[valid_rb.bidprice < valid_rb["rnd_bid"]]), \
      " with ", sum(auction), " impressions and ", sum(clicks_rand), " clicks \n" )

test_results_budget(auction, valid_rb.payprice, valid_rb.click)

The CTR for random bidding on validation set is for upper bound at  10  is  0.1112276353 %
The CPC for random bidding on validation set is for upper bound at  10  is  95139.7714286
Total cost for valid set is  3329892  with  31467  impressions and  35  clicks 

Sum payed for the clicked ones  6320  successful clicks  0  out of  56
Cost per Click  inf
CTR  0.0


## Linear model

In [15]:
#Define features
features = ["weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X and y

train_df['area'] = train_df['slotwidth']*train_df['slotheight']


X_train = train_df[features]
y_train = train_df['click']

In [16]:
# Train the model to estimate pCTR
clf = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5, objective= 'binary:logistic')
clf.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [17]:
# preprocess validation set
valid_df['area'] = valid_df['slotwidth']*valid_df['slotheight']

X_valid = valid_df[features]

In [18]:
## predicts probabilities of being clicked for validation
y_p_valid = clf.predict(X_valid)

avgCTR is the average of probability that the ad will be clicked

In [19]:
avgCTR = sum(y_p_valid)/len(y_p_valid)
print("Average CTR ", avgCTR)

Average CTR  0.000755362039884


In [20]:
def calculate_bid(base_bid, y_pred, avgCTR):
    bid = np.multiply(y_pred, base_bid/avgCTR)
    return bid

In [21]:
def max_bid_base(prices, y_p_valid, avgCTR):
    ration = np.zeros(len(prices))
    total = len(valid_df)

    for i in range(len(prices)):
        tmp = calculate_bid(prices[i], y_p_valid, avgCTR) # calculates our bid price
        tmp_2 = valid_df.click[valid_df.bidprice < tmp] # stores 0/1 for those auctions we won
        if len(tmp_2) == 0:
            ration[i] = 0
        else:
            ration[i] = sum(tmp_2)/len(tmp_2) # clicked/won actions

    print("The bid_base with best CTR: ", prices[ration.argmax()])
    print("CTR for winning bid_base is ", ration.max())
    return prices[ration.argmax()]

In [22]:
prices = range(1, 301, 1)

bid_base = max_bid_base(prices, y_p_valid, avgCTR)

predicted_bid_v = calculate_bid(bid_base, y_p_valid, avgCTR)
impressions_c = valid_df.click[valid_df.bidprice < predicted_bid_v]
total_cost_c = sum(valid_df.payprice[valid_df.bidprice < predicted_bid_v])

print("Cost for valid set ", total_cost_c, ", CPC ", (total_cost_c/sum(impressions_c)), " impressions ",\
      len(impressions_c)," clicks ", sum(impressions_c))

The bid_base with best CTR:  20
CTR for winning bid_base is  0.0128479657388
Cost for valid set  40386 , CPC  6731.0  impressions  467  clicks  6


In [23]:
our_bids = calculate_bid(bid_base, y_p_valid, avgCTR) # calculates our bid price
price_payed = 0
clicked_won = 0 
won = 0

for i in range(len(our_bids)): 
    if (valid_df.bidprice[i] <= our_bids[i]):
        price_payed = price_payed + valid_df.payprice[i]
        if (price_payed < 6250):       
            won = won + 1
            if (valid_df.click[i] == 1):
                clicked_won = clicked_won + 1
        else:
            break
        
print("With budget 6250 we won ", won, " auctions and on ", clicked_won, "of them was clicked" )
print("CPC is then ", (price_payed/clicked_won))

With budget 6250 we won  66  auctions and on  1 of them was clicked
CPC is then  6281.0
